In [34]:
import pandas as pd
from citipy import citipy

import csv
from datetime import datetime
import json
from pandas.io.json import json_normalize

#import data
weather = pd.read_json("../ow.json") #1/1/2013-1/1/2020 hourly weather data
complaints = pd.read_csv("../complaints.csv", encoding = "ISO-8859-1")

In [35]:
crime = pd.read_csv("../crime.csv")

In [36]:
import numpy as np
crime_types = crime['OFFENSE_TYPE_ID'].unique()


In [37]:
drop_list = ['GEO_X', 'GEO_Y', 'DISTRICT_ID', 
             'REPORTED_DATE', 'LAST_OCCURRENCE_DATE',
            'INCIDENT_ID', 'OFFENSE_ID', 'OFFENSE_CODE_EXTENSION']
crime.drop(columns=drop_list, inplace=True)


In [32]:
crime.head()
crime['']

,OFFENSE_CODE,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,FIRST_OCCURRENCE_DATE,INCIDENT_ADDRESS,GEO_LON,GEO_LAT,PRECINCT_ID,NEIGHBORHOOD_ID
0,5213,weapon-unlawful-discharge-of,all-other-crimes,6/15/2016 11:31:00 PM,NaN,-104.809881,39.773188,521,montbello
1,2399,theft-other,larceny,10/11/2017 12:30:00 PM,NaN,-104.781434,39.785649,522,gateway-green-valley-ranch
2,2305,theft-items-from-vehicle,theft-from-motor-vehicle,3/4/2016 8:00:00 PM,2932 S JOSEPHINE ST,-104.957381,39.663490,314,wellshire
3,2399,theft-other,larceny,1/30/2018 7:20:00 PM,705 S COLORADO BLVD,-104.941440,39.702698,312,belcaro
4,2303,theft-shoplift,larceny,6/22/2017 8:53:00 PM,2810 E 1ST AVE,-104.955370,39.717107,311,cherry-creek


In [20]:
crime[['Date','Hour', 'am/pm']] = crime.FIRST_OCCURRENCE_DATE.str.split(expand=True)

In [26]:
crime = crime.rename(columns = {'OFFENSE_CODE':'Code', 
                               'OFFENSE_TYPE_ID':'Offense_Type', 
                               'INCIDENT_ADDRESS':'Address', 
                               'GEO_LON':'Lon',
                               'GEO_LAT':'Lat', 
                               'PRECINCT_ID':'Precint', 
                               'NEIGHBORHOOD_ID':'Neighborhood'})

In [27]:
crime.head()

,Code,Offense_Type,OFFENSE_CATEGORY_ID,FIRST_OCCURRENCE_DATE,Address,Lon,Lat,Precint,Neighborhood,Date,Hour,am/pm
0,5213,weapon-unlawful-discharge-of,all-other-crimes,6/15/2016 11:31:00 PM,NaN,-104.809881,39.773188,521,montbello,6/15/2016,11:31:00,PM
1,2399,theft-other,larceny,10/11/2017 12:30:00 PM,NaN,-104.781434,39.785649,522,gateway-green-valley-ranch,10/11/2017,12:30:00,PM
2,2305,theft-items-from-vehicle,theft-from-motor-vehicle,3/4/2016 8:00:00 PM,2932 S JOSEPHINE ST,-104.957381,39.663490,314,wellshire,3/4/2016,8:00:00,PM
3,2399,theft-other,larceny,1/30/2018 7:20:00 PM,705 S COLORADO BLVD,-104.941440,39.702698,312,belcaro,1/30/2018,7:20:00,PM
4,2303,theft-shoplift,larceny,6/22/2017 8:53:00 PM,2810 E 1ST AVE,-104.955370,39.717107,311,cherry-creek,6/22/2017,8:53:00,PM


In [13]:
#expand list from columns into new databases
weather_temp = (json_normalize(weather['main']))
weather_wind = json_normalize(weather['wind'])
weather_clouds = json_normalize(weather['clouds'])

In [14]:
#create new columns for desired fields from expanded list
weather_temp.head()
weather['Temp'] = weather_temp['temp']
weather['Temp Max'] = weather_temp['temp_max']
weather['Wind'] = weather_wind.speed
weather['Clouds'] = weather_clouds
#split date iso into columns, then delete unwanted columns 
weather[['Date','Hour', 'Trash', 'Trash2']] = weather.dt_iso.str.split(expand=True)
columns = ['wind', 'clouds', 'weather','main', 'timezone', 'snow', 'rain', 'dt', 'Trash', 'Trash2', 'dt_iso']
weather.drop(columns=columns, inplace=True)

In [15]:
weather.head()

,city_name,lat,lon,Temp,Temp Max,Wind,Clouds,Date,Hour
0,Denver,39.739236,-104.984862,20.88,25.00,2.39,75,2013-01-01,00:00:00
1,Denver,39.739236,-104.984862,19.08,23.00,3.36,75,2013-01-01,01:00:00
2,Denver,39.739236,-104.984862,16.56,21.00,5.82,75,2013-01-01,02:00:00
3,Denver,39.739236,-104.984862,14.41,18.00,8.05,75,2013-01-01,03:00:00
4,Denver,39.739236,-104.984862,16.30,19.99,4.70,75,2013-01-01,04:00:00


In [145]:
#create dataframe for speends over 40mph 
wind_df = weather.loc[weather.Wind > 40]
wind_df

,city_name,lat,lon,Temp,Temp Max,Wind,Clouds,Date,Hour
5162,Denver,39.739236,-104.984862,60.98,64.00,47.20,90,2013-08-04,02:00:00
10599,Denver,39.739236,-104.984862,33.76,37.00,40.26,90,2014-03-18,14:00:00
11611,Denver,39.739236,-104.984862,49.96,51.80,43.84,75,2014-04-29,18:00:00
46395,Denver,39.739236,-104.984862,52.65,54.14,48.32,75,2018-04-17,22:00:00
46397,Denver,39.739236,-104.984862,46.90,48.20,40.26,20,2018-04-18,00:00:00
54312,Denver,39.739236,-104.984862,30.47,33.01,46.08,90,2019-03-13,18:00:00
59070,Denver,39.739236,-104.984862,64.54,68.00,40.26,75,2019-09-28,00:00:00
59358,Denver,39.739236,-104.984862,63.72,72.00,50.78,75,2019-10-10,00:00:00


In [160]:
#create dataframe for temps under 10f 
cold_df = weather.loc[weather['Temp Max'] < 10]
cold_df_new = cold_df.set_index('Date')


,city_name,lat,lon,Temp,Temp Max,Wind,Clouds,Hour
Date,,,,,,,,
2013-01-12,Denver,39.739236,-104.984862,4.10,8.60,9.17,40,08:00:00
2013-01-12,Denver,39.739236,-104.984862,5.90,8.60,8.05,75,09:00:00
2013-01-12,Denver,39.739236,-104.984862,4.10,6.80,5.82,40,10:00:00
2013-01-12,Denver,39.739236,-104.984862,5.00,5.00,3.36,1,11:00:00
2013-01-12,Denver,39.739236,-104.984862,4.10,5.00,3.36,90,12:00:00
...,...,...,...,...,...,...,...,...
2019-10-30,Denver,39.739236,-104.984862,6.73,9.00,4.70,0,12:00:00
2019-10-30,Denver,39.739236,-104.984862,6.42,9.00,3.36,0,13:00:00
2019-10-30,Denver,39.739236,-104.984862,6.17,8.01,3.36,100,14:00:00
